In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import sys
from collections import Counter
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import multiprocessing
from multiprocessing import Pool
import itertools
from copy import deepcopy
import random

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.decomposition import PCA

from keras.models import Sequential, Model , load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, PReLU, GlobalMaxPooling2D, ReLU
from keras.optimizers import Adam, Optimizer
from keras import backend as K
from keras.utils import to_categorical, Sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.applications.resnet_v2 import ResNet50V2, ResNet152V2
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet121, DenseNet169
from keras.engine import Layer
#import tensorflow as tf
from skimage.transform import AffineTransform, warp

from datetime import datetime
import pickle
import scipy.stats as stats
import os
import sklearn.metrics
import tensorflow_hub as hub

from efficientnet.keras import EfficientNetB0
import albumentations
from albumentations.core.transforms_interface import DualTransform, ImageOnlyTransform
from albumentations.augmentations import functional as F

from PIL import Image
from PIL import ImageOps

from utils import *

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
y = pd.read_csv("data/train.csv")
yEval = pd.read_csv("data/test.csv")
classMap = pd.read_csv("data/class_map.csv")

In [ ]:
labels = ["grapheme_root","vowel_diacritic","consonant_diacritic"]

In [ ]:
y = y.set_index("image_id")

In [ ]:
tables = [pq.read_table('data/train_image_data_{0}.parquet'.format(i)) for i in range(4)]
tables = [table.to_pandas() for table in tables]
df = pd.concat(tables)
df = df.set_index("image_id")
del tables

In [ ]:
size=(128, 128)

In [ ]:
class DataLoader(Sequence):
    def __init__(self,X,y,training,batch_size=64,size=(255,255),alpha=3,transform=None,trim="edge"):
        self.training = training
        self.batch_size=batch_size
        self.X=X
        self.y=y
        self.size=size
        self.alpha=alpha
        self.transform=transform
        self.trim=trim

    def __len__(self):
        return int(np.ceil(self.X.shape[0] / self.batch_size))


    def __getitem__(self, idx):
        
        _imgs=self.X[idx * self.batch_size:(idx + 1) * self.batch_size,:,:]
        
        imgs=[]
        for img in _imgs:
            imgs.append(transformImg(img,size=self.size,training=self.training,trim=self.trim))
        
        if self.transform is not None:
            _imgs=[]
            for img in imgs:
                if np.random.random()<0.5:
                    res = self.transform(image=img)
                    img = res['image'].astype(np.float32)
                else:
                    pass
                _imgs.append(img)
            imgs=_imgs 
        
        imgs=np.asarray(imgs)
        
        
        ret_y=[]
        for label in labels:
            ret_y.append(to_categorical(self.y[idx * self.batch_size:(idx + 1) * self.batch_size][label],num_classes=len(set(y[label]))))
    
        #mix up
        if self.training :
            r = np.random.permutation(imgs.shape[0])
            imgs2=deepcopy(imgs)[r]
            grapheme=ret_y[0]
            vowel=ret_y[1]
            consonant=ret_y[2]
            grapheme2=deepcopy(grapheme)[r]
            vowel2=deepcopy(vowel)[r]
            consonant2=deepcopy(consonant)[r]
            ratio=np.random.beta(self.alpha,self.alpha,imgs.shape[0])
            ratio[ratio>1]=1
            ratio[ratio<0]=0
            imgs=np.tile(ratio,(3,*size,1)).T*imgs+np.tile((1-ratio),(3,*size,1)).T*imgs2
            grapheme=np.tile(ratio,(168,1)).T*grapheme+np.tile((1-ratio),(168,1)).T*grapheme2
            vowel=np.tile(ratio,(11,1)).T*vowel+np.tile((1-ratio),(11,1)).T*vowel2
            consonant=np.tile(ratio,(7,1)).T*consonant+np.tile((1-ratio),(7,1)).T*consonant2
            grapheme=grapheme.astype(np.float32)
            vowel=vowel.astype(np.float32)
            consonant=consonant.astype(np.float32)
            ret_y=[grapheme,vowel,consonant]
   
        if self.training:
            imgs = [cutout(img) for img in imgs]
            pass
            
        imgs = np.asarray(imgs).astype(np.float32)/255.0
            

        return imgs, ret_y

def transformImg(img,size=(255,255),training=True,clahe=False,mergin=5,trim="edge"):
    ret2, img = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)
    img = 255-img
    if training:
        img=affine_image(img)
    if trim=="edge":
        img = img[upper(img,mergin):lower(img,mergin),lefter(img,mergin):righter(img,mergin)]
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = cv2.resize(img, (size[0],size[1]))
    kernel = np.ones((3,3),np.float32)/9
    img = cv2.filter2D(img,-1,kernel)
    return img

def cutout(image_origin):
    # 最後に使うfill()は元の画像を書き換えるので、コピーしておく
    image = np.copy(image_origin)
    mask_value = image.mean()

    h, w, c = image.shape
    # マスクをかける場所のtop, leftをランダムに決める
    # はみ出すことを許すので、0以上ではなく負の値もとる(最大mask_size // 2はみ出す)
    mask_size=h//2
    top = np.random.randint(0 - mask_size // 2, h - mask_size)
    left = np.random.randint(0 - mask_size // 2, w - mask_size)
    bottom = top + mask_size
    right = left + mask_size

    # はみ出した場合の処理
    if top < 0:
        top = 0
    if left < 0:
        left = 0

    # マスク部分の画素値を平均値で埋める
    image[top:bottom, left:right, :].fill(mask_value)
    return image

In [ ]:
def getMultiBased():
    model =  Xception(weights="imagenet", include_top=False, input_shape=(*size,3))
    x = model.output  
    x = GlobalAveragePooling2D()(x)
    x = Dropout(rate=0.7)(x)
    #x = Dense(1024)(x)
    #grapheme = PReLU()(x)
    #grapheme = Dropout(0.5)(grapheme)
    grapheme = Dense(168, activation="softmax")(x)
    #vowel = ReLU()(x)
    #vowel = Dropout(0.5)(vowel)
    vowel = Dense(11,activation="softmax")(x)
    #consonant = ReLU()(x)
    #consonant = Dropout(0.5)(consonant)
    consonant = Dense(7, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=[grapheme,vowel,consonant])
    return model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.values.reshape(-1,137,236), y, train_size=0.9, random_state=6666)
del df

In [ ]:
early_stopping =  EarlyStopping(monitor='val_loss', min_delta=0.0, patience=20)
checkpoint = ModelCheckpoint(filepath="tmp-xce-justresize-epoch{epoch:04}.h5")
schedule = ReduceLROnPlateau(patience=5,verbose=1)
tensorboard=TensorBoard(log_dir='./logs')

In [ ]:
batch_size = 256
train_gen = DataLoader(X_train, y_train, training=True, batch_size=batch_size,size=size,alpha=1.0,trim=None)
valid_gen = DataLoader(X_test, y_test, training=False, batch_size=batch_size,size=size,trim=None)
#strategy = tf.distribute.MirroredStrategy()
#print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
#with strategy.scope():
model = getMultiBased()
model.summary()
model.compile(optimizer=Adam(), metrics=["acc"], loss="categorical_crossentropy", loss_weights=[0.5,0.25,0.25])
print("compiled")
model.fit_generator(train_gen, validation_data=valid_gen, epochs=70, workers=16, use_multiprocessing=True, callbacks=[checkpoint,schedule,tensorboard])
model.save("multiXception11-justresize.h5", include_optimizer=False)

In [ ]:
model.evaluate(valid_gen)